In [1]:
from typing import Union, List, Dict

from utils import logger
logger.fmt_string = "%(name)s: %(message)s"

from backtesting.data import OrderStatus, OrderRequest
from backtesting.strategy import Strategy
from utils.data import Trade, OrderBook
from backtesting import readers, backtest
from backtesting.output import StorageOutput
from backtesting.readers import OrderbookReader

from metrics.filters import Filters
from metrics.metrics import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
class GatlingMM(Strategy):
  def __init__(self, side_volume=30000):
    super().__init__()
    self.side_volume = side_volume
    self.volumes_left = {} # add cancellation policy

  def define_orders(self, row: Union[Trade, OrderBook],
                    statuses: List[OrderStatus],
                    memory: Dict[str, Union[Trade, OrderBook]]) -> List[OrderRequest]:
    if self.balance.get(row.symbol, None) is None:
      if isinstance(row, OrderBook) and self.balance.get(row.symbol, None) is None:
        # Initialize first orders
        ask_volume = min(self._get_allowed_volume(row.symbol, memory, 'ask'), self.side_volume)
        self.volumes_left[(row.symbol, 'ask')] = self.side_volume - ask_volume
        ask_order = OrderRequest.create_ask(row.ask_prices[0], ask_volume, row.symbol, row.timestamp)

        bid_volume = min(self._get_allowed_volume(row.symbol, memory, 'bid'), self.side_volume)
        self.volumes_left[(row.symbol, 'bid')] = self.side_volume - bid_volume
        bid_order = OrderRequest.create_bid(row.bid_prices[0], bid_volume, row.symbol, row.timestamp)

        return [ask_order, bid_order]
    elif self.balance.get(row.symbol, None) is not None:
      orders = []
      for status in statuses:
        order: OrderRequest = self.active_orders[status.id]

        if status.status != 'partial':
          self.volumes_left[(order.symbol, order.side)] += order.volume - order.volume_filled
        elif status.status == 'partial':
          self.volumes_left[(order.symbol, order.side)] += status.volume

        volume = min(self.volumes_left[(order.symbol, order.side)], self._get_allowed_volume(order.symbol, memory, order.side))
        if volume > 0:
          self.volumes_left[(order.symbol, order.side)] -= volume

          if order.side == 'bid':
            price = memory[('orderbook', order.symbol)].bid_prices[0]
          else:
            price = memory[('orderbook', order.symbol)].ask_prices[0]

          neworder = OrderRequest.create(price, volume, order.symbol, order.side, row.timestamp)
          orders.append(neworder)

      return orders
    return []

In [3]:
reader = OrderbookReader('../tests/resources/huge_dataset/orderbook_10_03_20.csv.gz', 
                         '../tests/resources/huge_dataset/trades_10_03_20.csv.gz',
                         stop_after=300000, depth_to_load=5)

output = StorageOutput([], [])
output.balances = []
strategy = GatlingMM()
strategy.balance_listener = lambda b, t: output.balances.append((t, b))

backtester = backtest.Backtest(reader, strategy, delay=300)


<backtest>: Initialized <Backtest with reader=<orderbook-reader on orderbook_file=../tests/resources/huge_dataset/orderbook_10_03_20.csv.gz, trades_file=../tests/resources/huge_dataset/trades_10_03_20.csv.gz, batch_nrows=10000>>


In [4]:
backtester.run()

<backtest>: Backtest initialize run
<Strategy>: New order: <order-request id=0, command=new, symbol=XBTUSD, side=ask, volume=30000, price=7823.0>
<Strategy>: New order: <order-request id=1, command=new, symbol=XBTUSD, side=bid, volume=30000, price=7822.5>
<Strategy>: New order: <order-request id=2, command=new, symbol=ETHUSD, side=ask, volume=30000, price=199.05>
<Strategy>: New order: <order-request id=3, command=new, symbol=ETHUSD, side=bid, volume=12138.449999999999, price=199.0>
<Strategy>: Received status: <order-status id=1, status=partial, volume=2038, at=2020-03-10 18:25:35.703000>
<Strategy>: Received status: <order-status id=1, status=partial, volume=3130, at=2020-03-10 18:25:35.703000>
<Strategy>: Received status: <order-status id=1, status=partial, volume=3138, at=2020-03-10 18:25:35.703000>
<Strategy>: Received status: <order-status id=1, status=partial, volume=10089, at=2020-03-10 18:25:35.703000>
<Strategy>: Received status: <order-status id=1, status=partial, volume=162

<Strategy>: Received status: <order-status id=8, status=partial, volume=8148, at=2020-03-10 18:25:37.255000>
<Strategy>: Received status: <order-status id=13, status=partial, volume=4539, at=2020-03-10 18:25:37.255000>
<Strategy>: Received status: <order-status id=8, status=partial, volume=8398, at=2020-03-10 18:25:37.255000>
<Strategy>: Received status: <order-status id=13, status=partial, volume=4619, at=2020-03-10 18:25:37.255000>
<Strategy>: Received status: <order-status id=8, status=partial, volume=8478, at=2020-03-10 18:25:37.255000>
<Strategy>: Received status: <order-status id=13, status=partial, volume=5006, at=2020-03-10 18:25:37.255000>
<Strategy>: Received status: <order-status id=8, status=partial, volume=8865, at=2020-03-10 18:25:37.255000>
<Strategy>: Received status: <order-status id=13, status=partial, volume=5036, at=2020-03-10 18:25:37.255000>
<Strategy>: Received status: <order-status id=8, status=partial, volume=8895, at=2020-03-10 18:25:37.255000>
<Strategy>: Rec

<Strategy>: Received status: <order-status id=25, status=partial, volume=8499, at=2020-03-10 18:25:38.481000>
<Strategy>: Received status: <order-status id=14, status=finished, volume=-1, at=2020-03-10 18:25:38.481000>
<Strategy>: Received status: <order-status id=15, status=partial, volume=435, at=2020-03-10 18:25:38.481000>
<Strategy>: Received status: <order-status id=10, status=partial, volume=4295, at=2020-03-10 18:25:38.481000>
<Strategy>: New order: <order-request id=69, command=new, symbol=XBTUSD, side=bid, volume=1860, price=7816.5>
<Strategy>: New order: <order-request id=70, command=new, symbol=XBTUSD, side=bid, volume=5721, price=7816.5>
<Strategy>: New order: <order-request id=71, command=new, symbol=XBTUSD, side=bid, volume=1936, price=7816.5>
<Strategy>: New order: <order-request id=72, command=new, symbol=XBTUSD, side=bid, volume=5796, price=7816.5>
<Strategy>: New order: <order-request id=73, command=new, symbol=XBTUSD, side=bid, volume=5000, price=7816.5>
<Strategy>: 

<Strategy>: Received status: <order-status id=28, status=partial, volume=3385, at=2020-03-10 18:25:38.891000>
<Strategy>: Received status: <order-status id=19, status=partial, volume=679, at=2020-03-10 18:25:38.891000>
<Strategy>: Received status: <order-status id=28, status=finished, volume=-1, at=2020-03-10 18:25:38.891000>
<Strategy>: Received status: <order-status id=29, status=finished, volume=-1, at=2020-03-10 18:25:38.891000>
<Strategy>: Received status: <order-status id=30, status=finished, volume=-1, at=2020-03-10 18:25:38.891000>
<Strategy>: Received status: <order-status id=31, status=finished, volume=-1, at=2020-03-10 18:25:38.891000>
<Strategy>: Received status: <order-status id=32, status=finished, volume=-1, at=2020-03-10 18:25:38.891000>
<Strategy>: Received status: <order-status id=33, status=finished, volume=-1, at=2020-03-10 18:25:38.891000>
<Strategy>: Received status: <order-status id=34, status=partial, volume=1352, at=2020-03-10 18:25:38.891000>
<Strategy>: Recei

<Strategy>: Received status: <order-status id=71, status=partial, volume=506, at=2020-03-10 18:25:39.714000>
<Strategy>: Received status: <order-status id=36, status=finished, volume=-1, at=2020-03-10 18:25:39.714000>
<Strategy>: Received status: <order-status id=37, status=partial, volume=938, at=2020-03-10 18:25:39.714000>
<Strategy>: Received status: <order-status id=22, status=finished, volume=-1, at=2020-03-10 18:25:39.714000>
<Strategy>: Received status: <order-status id=23, status=partial, volume=419, at=2020-03-10 18:25:39.714000>
<Strategy>: Received status: <order-status id=71, status=finished, volume=-1, at=2020-03-10 18:25:39.714000>
<Strategy>: Received status: <order-status id=72, status=partial, volume=570, at=2020-03-10 18:25:39.714000>
<Strategy>: Received status: <order-status id=37, status=partial, volume=2938, at=2020-03-10 18:25:39.714000>
<Strategy>: Received status: <order-status id=23, status=partial, volume=2419, at=2020-03-10 18:25:39.714000>
<Strategy>: Recei

...

<Strategy>: Received status: <order-status id=647, status=partial, volume=535, at=2020-03-10 18:29:04.453000>
<Strategy>: Received status: <order-status id=508, status=partial, volume=312, at=2020-03-10 18:29:04.453000>
<Strategy>: New order: <order-request id=746, command=new, symbol=XBTUSD, side=bid, volume=535, price=7828.0>
<Strategy>: New order: <order-request id=747, command=new, symbol=XBTUSD, side=bid, volume=312, price=7828.0>
<Strategy>: Received status: <order-status id=647, status=finished, volume=-1, at=2020-03-10 18:29:04.494000>
<Strategy>: Received status: <order-status id=648, status=finished, volume=-1, at=2020-03-10 18:29:04.494000>
<Strategy>: Received status: <order-status id=649, status=partial, volume=1907, at=2020-03-10 18:29:04.494000>
<Strategy>: Received status: <order-status id=508, status=finished, volume=-1, at=2020-03-10 18:29:04.494000>
<Strategy>: Received status: <order-status id=509, status=finished, volume=-1, at=2020-03-10 18:29:04.494000>
<Strategy

<Strategy>: Received status: <order-status id=686, status=partial, volume=1273, at=2020-03-10 18:29:05.963000>
<Strategy>: New order: <order-request id=760, command=new, symbol=XBTUSD, side=bid, volume=1273, price=7826.0>
<Strategy>: Received status: <order-status id=391, status=finished, volume=-1, at=2020-03-10 18:29:06.209000>
<Strategy>: Received status: <order-status id=392, status=finished, volume=-1, at=2020-03-10 18:29:06.209000>
<Strategy>: Received status: <order-status id=393, status=finished, volume=-1, at=2020-03-10 18:29:06.209000>
<Strategy>: Received status: <order-status id=394, status=finished, volume=-1, at=2020-03-10 18:29:06.209000>
<Strategy>: Received status: <order-status id=403, status=finished, volume=-1, at=2020-03-10 18:29:06.209000>
<Strategy>: Received status: <order-status id=404, status=finished, volume=-1, at=2020-03-10 18:29:06.209000>
<Strategy>: Received status: <order-status id=407, status=finished, volume=-1, at=2020-03-10 18:29:06.209000>
<Strateg

<Strategy>: Received status: <order-status id=733, status=finished, volume=-1, at=2020-03-10 18:29:06.209000>
<Strategy>: Received status: <order-status id=734, status=finished, volume=-1, at=2020-03-10 18:29:06.209000>
<Strategy>: Received status: <order-status id=735, status=finished, volume=-1, at=2020-03-10 18:29:06.209000>
<Strategy>: Received status: <order-status id=736, status=finished, volume=-1, at=2020-03-10 18:29:06.209000>
<Strategy>: Received status: <order-status id=737, status=finished, volume=-1, at=2020-03-10 18:29:06.209000>
<Strategy>: Received status: <order-status id=738, status=finished, volume=-1, at=2020-03-10 18:29:06.209000>
<Strategy>: Received status: <order-status id=739, status=finished, volume=-1, at=2020-03-10 18:29:06.209000>
<Strategy>: Received status: <order-status id=740, status=finished, volume=-1, at=2020-03-10 18:29:06.209000>
<Strategy>: Received status: <order-status id=741, status=finished, volume=-1, at=2020-03-10 18:29:06.209000>
<Strategy>

<Strategy>: Received status: <order-status id=489, status=finished, volume=-1, at=2020-03-10 18:29:07.801000>
<Strategy>: Received status: <order-status id=490, status=finished, volume=-1, at=2020-03-10 18:29:07.801000>
<Strategy>: New order: <order-request id=761, command=new, symbol=XBTUSD, side=bid, volume=3504, price=7824.0>
<Strategy>: New order: <order-request id=762, command=new, symbol=XBTUSD, side=bid, volume=5504, price=7824.0>
<Strategy>: New order: <order-request id=763, command=new, symbol=XBTUSD, side=bid, volume=5528, price=7824.0>
<Strategy>: New order: <order-request id=764, command=new, symbol=XBTUSD, side=bid, volume=17, price=7824.0>
<Strategy>: New order: <order-request id=765, command=new, symbol=XBTUSD, side=bid, volume=42, price=7824.0>
<Strategy>: New order: <order-request id=766, command=new, symbol=XBTUSD, side=bid, volume=82, price=7824.0>
<Strategy>: New order: <order-request id=767, command=new, symbol=XBTUSD, side=bid, volume=125, price=7824.0>
<Strategy>

KeyboardInterrupt: 

In [5]:
strategy.balance

defaultdict(<function backtesting.strategy.Strategy.__init__.<locals>.<lambda>>,
            {'ETHUSD': -55.24206455452015,
             'USD': 322095.55000000005,
             'XBTUSD': 88.9857037429305})

In [6]:
backtester.memory[('orderbook', 'XBTUSD')].bid_prices[0] * strategy.balance['XBTUSD'] + \
    backtester.memory[('orderbook', 'ETHUSD')].bid_prices[0] * strategy.balance['ETHUSD'] + \
    strategy.balance['USD']

1007026.1236881495

In [5]:
backtester.memory[('orderbook', 'XBTUSD')].bid_prices[0], backtester.memory[('orderbook', 'XBTUSD')].ask_prices[0]

(7828.0, 7828.5)